In [ ]:
# 📊 Snowflake ML Demo: Model Observability & Monitoring

This notebook sets up comprehensive monitoring for your deployed ML model to track performance, detect data drift, and ensure production reliability.

## 🎯 What We're Building
- **Performance Monitoring**: Track accuracy, precision, recall over time
- **Data Drift Detection**: Monitor feature distribution changes
- **Prediction Quality**: Analyze prediction patterns and confidence
- **Automated Alerting**: Set up notifications for model degradation
- **Business Dashboards**: Create executive views of model health

## 🚨 Why Observability Matters
- **Early Warning**: Detect issues before they impact business
- **Model Reliability**: Ensure consistent performance in production
- **Regulatory Compliance**: Maintain audit trails for healthcare AI
- **Continuous Improvement**: Data-driven model enhancement

## 📋 Key Components
- **Drift Monitoring**: Statistical tests for feature changes
- **Performance Tracking**: Model metrics over time
- **Alert System**: Automated notifications for issues
- **Executive Dashboards**: Business-friendly monitoring views


In [ ]:
# Import required libraries for model observability
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lit, count, avg, stddev, min, max, abs
from snowflake.ml.model_monitoring import ModelMonitor
import datetime
import uuid

print("✅ Observability libraries imported successfully!")
print("📊 Ready for comprehensive model monitoring setup")


In [ ]:
# Get current session and set context
session = Session.builder.getOrCreate()

# Set context for monitoring setup
session.use_database("ADVERSE_EVENT_MONITORING")
session.use_schema("ML_MODELS")
session.use_warehouse("ADVERSE_EVENT_WH")

print("✅ Session configured for model observability")
print(f"📍 Database: {session.get_current_database()}")
print(f"📍 Schema: {session.get_current_schema()}")
print(f"📍 Warehouse: {session.get_current_warehouse()}")


In [ ]:
## 🏗️ Setting Up Monitoring Infrastructure

First, let's create the additional monitoring tables we need for comprehensive observability.


In [ ]:
-- Create additional monitoring tables
USE SCHEMA ML_MODELS;

-- Model performance tracking over time
CREATE TABLE IF NOT EXISTS MODEL_PERFORMANCE_TRACKING (
    tracking_id VARCHAR(50),
    model_id VARCHAR(50),
    model_name VARCHAR(100),
    evaluation_date TIMESTAMP,
    dataset_name VARCHAR(100),
    sample_size INTEGER,
    accuracy_score FLOAT,
    precision_score FLOAT,
    recall_score FLOAT,
    f1_score FLOAT,
    auc_score FLOAT,
    performance_status VARCHAR(20),
    notes VARCHAR(1000)
);

-- Data drift monitoring
CREATE TABLE IF NOT EXISTS DATA_DRIFT_MONITORING (
    drift_id VARCHAR(50),
    model_id VARCHAR(50),
    feature_name VARCHAR(100),
    monitoring_date TIMESTAMP,
    reference_mean FLOAT,
    current_mean FLOAT,
    reference_std FLOAT,
    current_std FLOAT,
    drift_score FLOAT,
    drift_threshold FLOAT,
    drift_detected BOOLEAN,
    drift_severity VARCHAR(20)
);

-- Prediction quality monitoring
CREATE TABLE IF NOT EXISTS PREDICTION_QUALITY_MONITORING (
    quality_id VARCHAR(50),
    model_id VARCHAR(50),
    monitoring_date TIMESTAMP,
    total_predictions INTEGER,
    high_confidence_predictions INTEGER,
    low_confidence_predictions INTEGER,
    avg_confidence_score FLOAT,
    prediction_distribution OBJECT,
    quality_status VARCHAR(20)
);

SELECT '✅ Monitoring infrastructure created' as STATUS;


In [ ]:
## 🔍 Data Drift Detection

Let's implement data drift detection by comparing current inference data with our reference training data.


In [ ]:
print("🔍 Implementing data drift detection...")

# Load reference training data and current inference data
session.use_schema("DEMO_ANALYTICS")

try:
    # Reference data (training set)
    training_data = session.table("PREPARED_HEALTHCARE_DATA")
    
    # Current data (inference set)
    current_data = session.table("NEW_PATIENT_INFERENCE_DATA")
    
    print(f"✅ Data loaded:")
    print(f"   • Training data: {training_data.count()} records")
    print(f"   • Current data: {current_data.count()} records")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    raise

# Define features to monitor for drift
numerical_features = ["AGE", "TOTAL_CLAIM_AMOUNT_SUM", "NUM_CLAIMS", "NUM_CONDITIONS", "NUM_MEDICATIONS"]

print(f"\n📊 Monitoring {len(numerical_features)} numerical features for drift:")
for feature in numerical_features:
    print(f"   • {feature}")

# Calculate drift for each feature
model_id = "demo-model-id"  # In practice, get from MODEL_REGISTRY
drift_results = []
drift_threshold = 2.0  # 2 standard deviations

print(f"\n🔬 Calculating drift scores (threshold: {drift_threshold})...")

for feature in numerical_features:
    try:
        # Calculate reference statistics
        ref_stats = training_data.select(
            avg(col(feature)).alias("ref_mean"),
            stddev(col(feature)).alias("ref_std")
        ).collect()[0]
        
        # Calculate current statistics
        curr_stats = current_data.select(
            avg(col(feature)).alias("curr_mean"), 
            stddev(col(feature)).alias("curr_std")
        ).collect()[0]
        
        # Handle None values
        ref_mean = ref_stats["REF_MEAN"] or 0
        curr_mean = curr_stats["CURR_MEAN"] or 0
        ref_std = ref_stats["REF_STD"] or 1
        curr_std = curr_stats["CURR_STD"] or 1
        
        # Calculate drift score (normalized difference)
        drift_score = abs(curr_mean - ref_mean) / (ref_std + 1e-8)
        drift_detected = drift_score > drift_threshold
        
        # Determine severity
        if drift_score > 3.0:
            severity = "HIGH"
        elif drift_score > 2.0:
            severity = "MEDIUM"
        else:
            severity = "LOW"
            
        drift_results.append([
            str(uuid.uuid4()),  # drift_id
            model_id,
            feature,
            datetime.datetime.now(),
            ref_mean,
            curr_mean,
            ref_std,
            curr_std,
            drift_score,
            drift_threshold,
            drift_detected,
            severity
        ])
        
        # Print results
        status = "🚨 DRIFT DETECTED" if drift_detected else "✅ No drift"
        print(f"   {feature}: {status} (score: {drift_score:.3f})")
        
    except Exception as e:
        print(f"   ⚠️ Error processing {feature}: {e}")

print(f"\n📝 Saving drift monitoring results...")


In [ ]:
# Save drift monitoring results
if drift_results:
    session.use_schema("ML_MODELS")
    
    drift_df = session.create_dataframe(
        drift_results,
        schema=[
            "drift_id", "model_id", "feature_name", "monitoring_date",
            "reference_mean", "current_mean", "reference_std", "current_std",
            "drift_score", "drift_threshold", "drift_detected", "drift_severity"
        ]
    )
    
    drift_df.write.mode("append").save_as_table("DATA_DRIFT_MONITORING")
    
    print(f"✅ Drift monitoring results saved for {len(numerical_features)} features")
    
    # Summary statistics
    total_drift = sum(1 for result in drift_results if result[10])  # drift_detected column
    print(f"📊 Drift Summary:")
    print(f"   • Features monitored: {len(drift_results)}")
    print(f"   • Drift detected: {total_drift}")
    print(f"   • Drift rate: {total_drift/len(drift_results)*100:.1f}%")
    
else:
    print("⚠️ No drift results to save")


In [ ]:
## 📈 Prediction Quality Monitoring

Now let's analyze the quality and patterns of our model's predictions.


In [ ]:
print("📈 Analyzing prediction quality...")

session.use_schema("DEMO_ANALYTICS")

try:
    # Analyze predictions from AE_PREDICTIONS table
    predictions_stats = session.sql("""
        SELECT 
            COUNT(*) as total_predictions,
            COUNT(CASE WHEN PROBABILITY >= 0.7 THEN 1 END) as high_confidence_count,
            COUNT(CASE WHEN PROBABILITY < 0.7 THEN 1 END) as low_confidence_count,
            AVG(PROBABILITY) as avg_confidence,
            MAX(PROBABILITY) as max_confidence,
            MIN(PROBABILITY) as min_confidence,
            COUNT(CASE WHEN PREDICTED_AE LIKE '%High%' THEN 1 END) as high_risk_predictions,
            COUNT(CASE WHEN PREDICTED_AE LIKE '%Low%' THEN 1 END) as low_risk_predictions
        FROM AE_PREDICTIONS
    """).collect()
    
    if predictions_stats and predictions_stats[0]['TOTAL_PREDICTIONS'] > 0:
        stats = predictions_stats[0]
        
        total_preds = stats["TOTAL_PREDICTIONS"]
        high_confidence = stats["HIGH_CONFIDENCE_COUNT"] 
        low_confidence = stats["LOW_CONFIDENCE_COUNT"]
        avg_conf = stats["AVG_CONFIDENCE"]
        high_risk = stats["HIGH_RISK_PREDICTIONS"]
        low_risk = stats["LOW_RISK_PREDICTIONS"]
        
        print(f"✅ Prediction Quality Analysis:")
        print(f"   • Total predictions: {total_preds}")
        print(f"   • High confidence (≥0.7): {high_confidence} ({high_confidence/total_preds*100:.1f}%)")
        print(f"   • Low confidence (<0.7): {low_confidence} ({low_confidence/total_preds*100:.1f}%)")
        print(f"   • Average confidence: {avg_conf:.3f}")
        print(f"   • High risk predictions: {high_risk} ({high_risk/total_preds*100:.1f}%)")
        print(f"   • Low risk predictions: {low_risk} ({low_risk/total_preds*100:.1f}%)")
        
        # Determine quality status
        positive_rate = high_risk / total_preds if total_preds > 0 else 0
        
        if positive_rate > 0.8 or positive_rate < 0.1:
            quality_status = "WARNING"  # Extreme prediction bias
            quality_note = "Extreme prediction bias detected"
        elif avg_conf < 0.5:
            quality_status = "WARNING"  # Low confidence
            quality_note = "Low average confidence scores"
        elif total_preds < 10:
            quality_status = "INSUFFICIENT_DATA"
            quality_note = "Insufficient prediction volume"
        else:
            quality_status = "GOOD"
            quality_note = "Predictions within normal parameters"
            
        print(f"\n🎯 Quality Assessment: {quality_status}")
        print(f"   📝 {quality_note}")
        
        # Save quality monitoring results
        session.use_schema("ML_MODELS")
        session.sql(f"""
            INSERT INTO PREDICTION_QUALITY_MONITORING (
                quality_id, model_id, monitoring_date, total_predictions,
                high_confidence_predictions, low_confidence_predictions,
                avg_confidence_score, prediction_distribution, quality_status
            ) VALUES (
                '{str(uuid.uuid4())}', '{model_id}', CURRENT_TIMESTAMP(), {total_preds},
                {high_confidence}, {low_confidence}, {avg_conf}, 
                OBJECT_CONSTRUCT(
                    'high_risk', {high_risk}, 
                    'low_risk', {low_risk}, 
                    'positive_rate', {positive_rate},
                    'avg_confidence', {avg_conf}
                ),
                '{quality_status}'
            )
        """).collect()
        
        print(f"✅ Prediction quality results saved")
        
    else:
        print("⚠️ No predictions found for quality analysis")
        
except Exception as e:
    print(f"❌ Error analyzing prediction quality: {e}")


In [ ]:
## 📊 Creating Monitoring Dashboards

Let's create SQL views that provide executive-level monitoring dashboards.


In [ ]:
-- Create comprehensive monitoring dashboard views
USE SCHEMA ML_MODELS;

-- Executive Model Health Dashboard
CREATE OR REPLACE VIEW MODEL_HEALTH_DASHBOARD AS
SELECT 
    mr.model_name,
    mr.model_version,
    mr.accuracy_score,
    mr.f1_score,
    mr.model_status,
    mr.training_date,
    
    -- Drift indicators
    (SELECT COUNT(*) FROM DATA_DRIFT_MONITORING ddm 
     WHERE ddm.model_id = mr.model_id 
     AND ddm.drift_detected = TRUE 
     AND ddm.monitoring_date > DATEADD(day, -7, CURRENT_TIMESTAMP())) as features_with_drift_7d,
     
    -- Quality indicators  
    (SELECT quality_status FROM PREDICTION_QUALITY_MONITORING pqm 
     WHERE pqm.model_id = mr.model_id 
     ORDER BY pqm.monitoring_date DESC LIMIT 1) as latest_quality_status,
     
    -- Prediction volume
    (SELECT total_predictions FROM PREDICTION_QUALITY_MONITORING pqm 
     WHERE pqm.model_id = mr.model_id 
     ORDER BY pqm.monitoring_date DESC LIMIT 1) as daily_prediction_volume,
     
    -- Overall health score
    CASE 
        WHEN mr.f1_score >= 0.8 AND 
             (SELECT COUNT(*) FROM DATA_DRIFT_MONITORING ddm 
              WHERE ddm.model_id = mr.model_id AND ddm.drift_detected = TRUE 
              AND ddm.monitoring_date > DATEADD(day, -7, CURRENT_TIMESTAMP())) = 0
        THEN 'HEALTHY 🟢'
        WHEN mr.f1_score >= 0.6 
        THEN 'WARNING 🟡'
        ELSE 'CRITICAL 🔴'
    END as overall_health
    
FROM MODEL_REGISTRY mr
ORDER BY mr.training_date DESC;

-- View the dashboard
SELECT * FROM MODEL_HEALTH_DASHBOARD;


In [ ]:
## 🚨 Setting Up Automated Alerting

Now let's create an automated alerting system for model performance issues.


In [ ]:
-- Create alerting system infrastructure
USE SCHEMA ML_MODELS;

-- Alert configuration table
CREATE TABLE IF NOT EXISTS ALERT_CONFIGURATION (
    alert_id VARCHAR(50),
    alert_type VARCHAR(50),
    metric_name VARCHAR(100),
    threshold_value FLOAT,
    comparison_operator VARCHAR(10),
    notification_email VARCHAR(200),
    is_active BOOLEAN,
    created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

-- Insert default alert configurations
INSERT INTO ALERT_CONFIGURATION (alert_id, alert_type, metric_name, threshold_value, comparison_operator, notification_email, is_active) VALUES
('ALERT_001', 'PERFORMANCE', 'F1_SCORE', 0.6, '<', 'ml-team@company.com', TRUE),
('ALERT_002', 'DRIFT', 'DRIFT_SCORE', 2.0, '>', 'ml-team@company.com', TRUE),
('ALERT_003', 'QUALITY', 'POSITIVE_RATE', 0.9, '>', 'ml-team@company.com', TRUE),
('ALERT_004', 'QUALITY', 'POSITIVE_RATE', 0.05, '<', 'ml-team@company.com', TRUE),
('ALERT_005', 'VOLUME', 'DAILY_PREDICTIONS', 10, '<', 'ml-team@company.com', TRUE);

-- Alert history table
CREATE TABLE IF NOT EXISTS ALERT_HISTORY (
    alert_history_id VARCHAR(50),
    alert_id VARCHAR(50),
    model_id VARCHAR(50),
    alert_triggered_date TIMESTAMP,
    alert_message VARCHAR(1000),
    current_value FLOAT,
    threshold_value FLOAT,
    alert_status VARCHAR(20)
);

SELECT '✅ Alerting infrastructure created' as STATUS;


In [ ]:
-- Create comprehensive alert checking stored procedure
CREATE OR REPLACE PROCEDURE CHECK_MODEL_ALERTS()
RETURNS VARCHAR
LANGUAGE SQL
AS
$$
DECLARE
    alert_count INTEGER := 0;
    result_msg VARCHAR := '';
    drift_count INTEGER := 0;
    quality_issues INTEGER := 0;
BEGIN
    -- Check for performance alerts
    SELECT COUNT(*) INTO alert_count
    FROM MODEL_REGISTRY mr
    JOIN ALERT_CONFIGURATION ac ON ac.metric_name = 'F1_SCORE'
    WHERE mr.f1_score < ac.threshold_value 
    AND ac.is_active = TRUE;
    
    IF (alert_count > 0) THEN
        result_msg := 'PERFORMANCE ALERT: Model F1 score below threshold. ';
    END IF;
    
    -- Check for drift alerts
    SELECT COUNT(*) INTO drift_count
    FROM DATA_DRIFT_MONITORING ddm
    JOIN ALERT_CONFIGURATION ac ON ac.metric_name = 'DRIFT_SCORE'
    WHERE ddm.drift_score > ac.threshold_value
    AND ddm.monitoring_date > DATEADD(day, -1, CURRENT_TIMESTAMP())
    AND ac.is_active = TRUE;
    
    IF (drift_count > 0) THEN
        result_msg := result_msg || 'DRIFT ALERT: ' || drift_count || ' features showing drift. ';
    END IF;
    
    -- Check for prediction quality issues
    SELECT COUNT(*) INTO quality_issues
    FROM PREDICTION_QUALITY_MONITORING pqm
    WHERE pqm.quality_status = 'WARNING'
    AND pqm.monitoring_date > DATEADD(day, -1, CURRENT_TIMESTAMP());
    
    IF (quality_issues > 0) THEN
        result_msg := result_msg || 'QUALITY ALERT: Prediction quality issues detected. ';
    END IF;
    
    -- Final result
    IF (result_msg = '') THEN
        result_msg := '✅ All monitoring checks passed - model is healthy';
    ELSE
        result_msg := '🚨 ALERTS DETECTED: ' || result_msg;
    END IF;
    
    RETURN result_msg;
END;
$$;

-- Test the alert system
CALL CHECK_MODEL_ALERTS();


In [ ]:
## ✅ Model Observability Complete!

Your comprehensive ML monitoring system is now operational:

### 🎯 **Monitoring Capabilities Deployed**
- ✅ **Data Drift Detection**: Statistical monitoring of feature distributions
- ✅ **Prediction Quality Analysis**: Pattern detection in model outputs  
- ✅ **Performance Tracking**: Historical model metrics and trends
- ✅ **Executive Dashboards**: Business-friendly health monitoring
- ✅ **Automated Alerting**: Proactive issue detection and notification

### 📊 **Key Monitoring Views**
- **MODEL_HEALTH_DASHBOARD**: Executive summary of model status
- **DATA_DRIFT_MONITORING**: Feature-level drift detection results
- **PREDICTION_QUALITY_MONITORING**: Output quality and distribution analysis
- **ALERT_CONFIGURATION**: Customizable alerting thresholds

### 🚨 **Alerting System**
- **Performance Alerts**: F1 score degradation detection
- **Drift Alerts**: Feature distribution change notifications
- **Quality Alerts**: Prediction bias and confidence warnings
- **Volume Alerts**: Prediction frequency monitoring
- **Custom Thresholds**: Configurable alerting parameters

### 🏥 **Healthcare-Specific Benefits**
- **Patient Safety**: Early detection of model reliability issues
- **Regulatory Compliance**: Audit trails for AI decision making
- **Clinical Confidence**: Transparent model performance metrics
- **Risk Management**: Proactive intervention triggers

### 📈 **Business Value**
- **Reduced Downtime**: Early warning prevents model failures
- **Trust & Compliance**: Transparent AI for healthcare regulations
- **Continuous Improvement**: Data-driven model enhancement
- **Operational Excellence**: Automated monitoring reduces manual effort

### 🎯 **Sample Monitoring Queries**
```sql
-- Check current model health
SELECT * FROM MODEL_HEALTH_DASHBOARD;

-- Review recent drift detections
SELECT * FROM DATA_DRIFT_MONITORING 
WHERE drift_detected = TRUE 
ORDER BY monitoring_date DESC;

-- Run alert checks
CALL CHECK_MODEL_ALERTS();
```

## 📋 Next Steps
1. **Schedule Monitoring**: Set up automated runs of drift detection and alerting
2. **Customize Thresholds**: Adjust alert thresholds based on business requirements
3. **Integration**: Connect alerts to existing incident management systems
4. **Complete Demo**: Use `08_Demo_Walkthrough` for end-to-end demonstration

---
*Comprehensive ML observability ensures reliable, trustworthy AI in production healthcare environments.*
